# Credit Risk Resampling Techniques

In [83]:
import warnings
warnings.filterwarnings('ignore')

In [84]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [85]:
# Load the data
file_path = Path('lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [86]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]
X = pd.get_dummies(X)

In [87]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [88]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [89]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=22)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [90]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
stand_scale = StandardScaler()

In [91]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
stand_scale.fit(X_train)

StandardScaler()

In [92]:
# Scale the training and testing data
X_train_scaled = stand_scale.transform(X_train)
X_test_scaled = stand_scale.transform(X_test)

# Simple Logistic Regression

In [93]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [94]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9597791293213829

In [95]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  592,    48],
       [  102, 18642]], dtype=int64)

In [96]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.93      0.99      0.89      0.96      0.91       640
   low_risk       1.00      0.99      0.93      1.00      0.96      0.93     18744

avg / total       0.99      0.99      0.93      0.99      0.96      0.93     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [97]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
naive = RandomOverSampler(random_state=22)
X_resampled, y_resampled = naive.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56292, 'high_risk': 56292})

In [98]:
# Train the Logistic Regression model using the resampled data
new_model = LogisticRegression(solver='lbfgs', random_state=1)
new_model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [99]:
# Calculated the balanced accuracy score
new_prediction = new_model.predict(X_test_scaled)
balanced_accuracy_score(y_test, new_prediction)

0.9949697236448997

In [100]:
# Display the confusion matrix
confusion_matrix(y_test, new_prediction)

array([[  638,     2],
       [  130, 18614]], dtype=int64)

In [101]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, new_prediction))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.83      1.00      0.99      0.91      0.99      0.99       640
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18744

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [102]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_smote)

Counter({'low_risk': 56292, 'high_risk': 56292})

In [103]:
# Train the Logistic Regression model using the resampled data
model_3 = LogisticRegression(solver='lbfgs', random_state=1)
model_3.fit(X_smote, y_smote)

LogisticRegression(random_state=1)

In [104]:
# Calculated the balanced accuracy score
prediction_3 = model_3.predict(X_test_scaled)
balanced_accuracy_score(y_test, prediction_3)

0.9951030996585574

In [105]:
# Display the confusion matrix
confusion_matrix(y_test, prediction_3)

array([[  638,     2],
       [  125, 18619]], dtype=int64)

In [106]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      1.00      0.99      0.91      1.00      0.99       640
   low_risk       1.00      0.99      1.00      1.00      1.00      0.99     18744

avg / total       0.99      0.99      1.00      0.99      1.00      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [107]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 1860, 'low_risk': 1860})

In [108]:
# Train the Logistic Regression model using the resampled data
model_4 = LogisticRegression(solver='lbfgs', random_state=1)
model_4.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [109]:
# Calculate the balanced accuracy score
prediction_4 = model_4.predict(X_test_scaled)
balanced_accuracy_score(y_test, prediction_4)

0.8145119771660264

In [110]:
# Display the confusion matrix
confusion_matrix(y_test, prediction_4)

array([[  638,     2],
       [ 6895, 11849]], dtype=int64)

In [111]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.63      0.16      0.79      0.65       640
   low_risk       1.00      0.63      1.00      0.77      0.79      0.61     18744

avg / total       0.97      0.64      0.98      0.75      0.79      0.61     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [112]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

SMOTee = SMOTEENN(random_state=33)
x_resampled_teen, y_resampled_teen = SMOTee.fit_resample(X,y)
# View the count of target classes with Counter
Counter(y_resampled_teen)

Counter({'high_risk': 73887, 'low_risk': 74592})

In [113]:
# Train the Logistic Regression model using the resampled data
model_5 = LogisticRegression(solver='lbfgs', random_state=44)
model_5.fit(x_resampled_teen, y_resampled_teen)

LogisticRegression(random_state=44)

In [114]:
# Calculate the balanced accuracy score
prediction_5 = model_5.predict(X_test_scaled)
balanced_accuracy_score(y_test, prediction_5)

0.8074697236448997

In [115]:
# Display the confusion matrix
confusion_matrix(y_test, prediction_5)

array([[  638,     2],
       [ 7159, 11585]], dtype=int64)

In [116]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, prediction_5))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      1.00      0.62      0.15      0.78      0.64       640
   low_risk       1.00      0.62      1.00      0.76      0.78      0.59     18744

avg / total       0.97      0.63      0.98      0.74      0.78      0.59     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.


# Final Questions

1. Which model had the best balanced accuracy score?

Smote and random oversampler.

2. Which model had the best recall score?

Smote and random oversampler also had the top recall scores.

3. Which model had the best geometric mean score?

Smote and random oversampler were the most accurate model overall.